In [ ]:
from tkinter import *
from PIL import ImageTk, Image

In [ ]:
class ImageAnnotationGUI(object):
    
    def run(self):
        
        # window set-upI
        base_window = Tk()
        base_window.withdraw()
        self.base_window = base_window
        root = Toplevel()
        self.root = root
        
        #root.geometry('768x612')
        pad=3
        root.geometry("{0}x{1}+0+0".format(
        root.winfo_screenwidth()-pad, root.winfo_screenheight()-pad))
        
        root.title('Image Annotation Interface')
        f = Frame(root)
        f.pack(padx = 10, pady = 10)
        root.protocol ("WM_DELETE_WINDOW", self.destroy_all)
        
        # create QUIT button
        button = Button(f, text="QUIT", fg="red", command=self.destroy_all)
        button.grid(row=0, column=0, padx=10, pady=10, sticky=W)

        # create Hello button
        hi_there = Button(f, text="Hello", command=self.say_hi)
        hi_there.grid(row=0, column=1, padx=10, pady=10, sticky=W)
 
        # create horizontal scroll bar
        xscrollbar = Scrollbar(f, orient=HORIZONTAL)
        xscrollbar.grid(row=2, column=0, sticky=N+S+E+W, padx=10, pady=10)
        text = Text(f, wrap=NONE, xscrollcommand=xscrollbar.set, height=1, width=100)
        text.grid(row=1, column=0)
        xscrollbar.config(command=text.xview)
        scroll_contents = [i for i in range(200)]
        text.insert(END, scroll_contents)
        
        # create first label
        first_label = Label(f, text="First")
        first_label.grid(row=3, sticky=E)
        first_entry_box = Entry(f)
        first_entry_box.grid(row=3, column=1)
        
        # create first label save button
        first_save_button = Button(f, text="Save First Box", command=lambda: print(first_entry_box.get()))
        first_save_button.grid(row=3, column=2, padx=10, pady=10, sticky=W)
        
        # create second label
        second_label = Label(f, text="Second")
        second_label.grid(row=4, sticky=E)
        second_entry_box = Entry(f)
        second_entry_box.grid(row=4, column=1)
        
        # create second label save button
        second_save_button = Button(f, text="Save Second Box", command=lambda: print(second_entry_box.get()))
        second_save_button.grid(row=4, column=2, padx=10, pady=10, sticky=W)
        
        # first image
        width = 50
        height = 50
        img = Image.open("cat.png")
        img = img.resize((width,height), Image.ANTIALIAS)
        photoImg =  ImageTk.PhotoImage(img)
        w1 = Label(f, image=photoImg)
        w1.grid(row=5, column=0)
        
        # another scroll bar
        canvas=Canvas(f,bg='#FF00FF',width=300,height=300,scrollregion=(0,0,500,500))
        canvas.create_image(0, height, image=photoImg, anchor=NW)
        canvas.create_image(width*2, height, image=photoImg, anchor=NW)
        canvas.create_image(width*4, height, image=photoImg, anchor=NW)
        canvas.create_image(width*6, height, image=photoImg, anchor=NW)
        canvas.create_image(width*8, height, image=photoImg, anchor=NW)
        hbar=Scrollbar(f,orient=HORIZONTAL)
        hbar.grid(row=7, column=0, sticky=N+S+E+W, padx=10, pady=10)
        hbar.config(command=canvas.xview)
        canvas.config(width=300,height=300)
        canvas.config(xscrollcommand=hbar.set)
        canvas.grid(row=6, column=0, padx=10, pady=10)
        
        # run interface
        mainloop()
        
    def destroy_all(self):
        self.base_window.destroy()

        
    def say_hi(self):
        print("hi there, everyone!")

if __name__=='__main__':
    window = ImageAnnotationGUI()
    window.run()

In [1]:
from tkinter import *
from PIL import ImageTk, Image
import reload

ModuleNotFoundError: No module named 'reload'

In [103]:
class ImageAnnotationGUI(object):
    
    PADX = 10
    PADY = 10
    
    def __init__(self, controller):
        self.controller = controller
    
    def run(self):
        
        # construct window
        self.base_window = Tk() # must run this window, but contains no contents
        self.base_window.withdraw() # hide the window with no contents 
        self.root = Toplevel() # this is the main window (must use Toplevel to display images)
        base_window = self.base_window # makes code easier to read going onward
        root = self.root
        root.title('Image Annotation Interface')
        root.protocol ("WM_DELETE_WINDOW", self.destroy_all) # override 'X' button
        
        # open window in full screen mode
        screen_padding = 3
        screen_width = root.winfo_screenwidth()
        screen_height = root.winfo_screenheight()
        root.geometry("{0}x{1}+0+0".format(
            screen_width - screen_padding, 
            screen_height - screen_padding)
        )
        
        # create frame to hold contents
        f = Frame(root)
        f.pack(padx = self.PADX, pady = self.PADY)
        
        # first image
        width = 50
        height = 50
        img = Image.open("cat.png")
        img = img.resize((width,height), Image.ANTIALIAS)
        photoImg =  ImageTk.PhotoImage(img)
        w1 = Label(f, image=photoImg)
        w1.grid(row=1, column=1, sticky=W)
        
        # another scroll bar
        #canvas=Canvas(f,bg='#FF00FF',width=300,height=300,scrollregion=(0,0,500,500))
        #canvas.create_image(0, height, image=photoImg, anchor=NW)
        #canvas.create_image(width*2, height, image=photoImg, anchor=NW)
        #canvas.create_image(width*4, height, image=photoImg, anchor=NW)
        #canvas.create_image(width*6, height, image=photoImg, anchor=NW)
        #canvas.create_image(width*8, height, image=photoImg, anchor=NW)
        #hbar=Scrollbar(f,orient=HORIZONTAL)
        #hbar.grid(row=7, column=0, sticky=N+S+E+W, padx=10, pady=10)
        #hbar.config(command=canvas.xview)
        #canvas.config(width=300,height=300)
        #canvas.config(xscrollcommand=hbar.set)
        #canvas.grid(row=6, column=0, padx=10, pady=10)
        
        # scroll side view images
        #image_width, image_height = self.controller.get_image_dimensions()
        #number_of_images = self.controller.get_number_of_images()
        
        # TODO: scale images to appropriate size while maintaining aspect ratio
        # image_width = modify it to fit screen well
        # image_height = modify it to fit screen well
        #full_scroll_width = number_of_images*image_width
        #canvas = Canvas(f, bg='#FFFFFF', width=image_width+self.PADX, height=image_height+self.PADY, # do we want to see multiple images at a time?
        #                scrollregion=(0,0, full_scroll_width+self.PADX, image_height+self.PADY) )
        
        #width_offset = 0
        #for i in range(5):
        #    img = self.controller.get_side_image(filename = "cat.png")
        #    photoImg =  ImageTk.PhotoImage(img)
        #    canvas.create_image(width*width_offset, height, image=photoImg, anchor=NW)
        #    width_offset = width_offset + 2
        
        image_width, image_height = self.controller.get_image_dimensions()
        number_of_images = self.controller.get_number_of_images()
        full_scroll_width = number_of_images*image_width
        canvas = Canvas(f, bg='#FFFFFF', width=image_width, height=image_height, scrollregion=(0,0,full_scroll_width,image_height))
        
        canvas.create_image(0, 0, image=photoImg, anchor=NW)
        canvas.create_image(width, 0, image=photoImg, anchor=NW)
        canvas.create_image(width*2, 0, image=photoImg, anchor=NW)
        canvas.create_image(width*3, 0, image=photoImg, anchor=NW)
        canvas.create_image(width*4, 0, image=photoImg, anchor=NW)
        hbar=Scrollbar(f,orient=HORIZONTAL)
        hbar.grid(row=2, column=0, sticky=N+S+E+W, padx=10, pady=10)
        hbar.config(command=canvas.xview)
        canvas.config(xscrollcommand=hbar.set)
        canvas.grid(row=1, column=0, padx=10, pady=10, sticky=W)
        
        
        
        # run interface
        mainloop()
        
    def destroy_all(self):
        self.base_window.destroy()
        
    def say_hi(self):
        print("hi there, everyone!")

In [104]:
class OCTImages(object):
    
    def query_database(patient, visit, ID):
        return -1

In [105]:
# Controlller that makes calls to the database for us,
# and hopefully makes it so I won't have to make too many 
# changes to ImageAnnotationGUI as I'm figuring out how to 
# get the data from the database

# Also hopefully makes it so that we can easily trade 
# OCT images for another modality
class GUIController(object):
    
    # self.datasource = OCTImages
    
    # _init_(self, datasouce):
    #   self.datasource = datasource
    
    # def get_datasource():
    #   return self.datasource
    
    # def set_datasource(datasource):
    #   self.datasource = datasource
    
    # def get_top_image(patient, visit, left_or_right):
    #   datasource.query_database():
    
    # def get_side_image(patient, visit, left_or_right, time_stamp):
    #   datasource.query_database():
    
    # def get_patient_data(patient, visit, left_or_right, time_stamp):
    #   datasource.query_database():
    
    def __init__(self, model = OCTImages, view = ImageAnnotationGUI):
        self.model = model
        self.view = view
        
    def start(self):
        window = self.view(self)
        window.run()
        
    # method stub
    def get_top_image(self, filename = "cat.png", width = 50, height = 50):
        img = Image.open("cat.png")
        return img.resize((width, height), Image.ANTIALIAS)
    
    # method stub
    def get_side_image(self, filename = "cat.png", width = 50, height = 50):
        img = Image.open("cat.png")
        return img.resize((width, height), Image.ANTIALIAS)
    
    # method stub
    def get_number_of_images(self):
        return 5
    
    # method stub
    def get_image_dimensions(self):
        return 50,50

if __name__ == "__main__":
    gui_controller = GUIController()
    gui_controller.start()